In [48]:
import requests
from datetime import datetime

from api_key import api_key

In [25]:
url = "https://api.um.warszawa.pl/api/action/busestrams_get"

In [26]:
params = {
    "resource_id": "f2e5503e-927d-4ad3-9500-4ab9e55deb59",  # resource id given by the API
    "apikey": api_key,
    "type": 1,  # 1 - bus, 2 - tram
}

In [27]:
response = requests.get(url, params=params)

In [18]:
print(response.json())

{'result': [{'Lines': '219', 'Lon': 21.115034, 'VehicleNumber': '1000', 'Time': '2024-12-25 16:52:05', 'Lat': 52.234593, 'Brigade': '1'}, {'Lines': '225', 'Lon': 21.115663, 'VehicleNumber': '1001', 'Time': '2024-12-24 08:14:35', 'Lat': 52.234589, 'Brigade': '503'}, {'Lines': '213', 'Lon': 21.115089, 'VehicleNumber': '1002', 'Time': '2024-12-26 02:44:21', 'Lat': 52.234708, 'Brigade': '5'}, {'Lines': '213', 'Lon': 21.115123, 'VehicleNumber': '1003', 'Time': '2024-12-24 23:38:49', 'Lat': 52.234581, 'Brigade': '3'}, {'Lines': '219', 'Lon': 21.115253, 'VehicleNumber': '1004', 'Time': '2024-12-23 22:45:50', 'Lat': 52.234574, 'Brigade': '3'}, {'Lines': '213', 'Lon': 21.115905, 'VehicleNumber': '1005', 'Time': '2024-12-26 03:00:10', 'Lat': 52.234286, 'Brigade': '1'}, {'Lines': '311', 'Lon': 21.11533, 'VehicleNumber': '1006', 'Time': '2024-12-23 21:50:21', 'Lat': 52.234588, 'Brigade': '502'}, {'Lines': '213', 'Lon': 21.17877, 'VehicleNumber': '1007', 'Time': '2024-12-26 14:24:44', 'Lat': 52.184

In [19]:
print(len(response.json()["result"]))

1896


##### all are records recent?

In [51]:
how_long_ago = []

In [52]:
for bus in response.json()["result"]:
    lon = bus["Lon"]
    lat = bus["Lat"]
    lines = bus["Lines"]
    time = bus["Time"]
    number = bus["VehicleNumber"]
    brigade = bus["Brigade"]

    # parse time format: 2024-12-25 16:52:05
    time = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
    diff = time.now() - time
    how_long_ago.append(diff)

In [57]:
print(how_long_ago)

[datetime.timedelta(seconds=79577, microseconds=202252), datetime.timedelta(days=2, seconds=24227, microseconds=202277), datetime.timedelta(seconds=44041, microseconds=202290), datetime.timedelta(days=1, seconds=55173, microseconds=202301), datetime.timedelta(days=2, seconds=58352, microseconds=202313), datetime.timedelta(seconds=43092, microseconds=202324), datetime.timedelta(days=2, seconds=61681, microseconds=202335), datetime.timedelta(seconds=1897, microseconds=202346), datetime.timedelta(days=1, seconds=79202, microseconds=202357), datetime.timedelta(days=2, seconds=65632, microseconds=202368), datetime.timedelta(days=24, seconds=20996, microseconds=202379), datetime.timedelta(days=26, seconds=7565, microseconds=202390), datetime.timedelta(days=2, seconds=67511, microseconds=202401), datetime.timedelta(seconds=1899, microseconds=202411), datetime.timedelta(days=41, seconds=2788, microseconds=202422), datetime.timedelta(days=1, seconds=75356, microseconds=202433), datetime.timedel

In [61]:
quantiles = [10, 25, 30, 35, 40, 50, 75, 90, 99]
for q in quantiles:
    print(f"{q}th percentile: {sorted(how_long_ago)[int(q/100*len(how_long_ago))]}")

10th percentile: 0:31:36.214447
25th percentile: 0:31:39.212259
30th percentile: 0:31:42.207569
35th percentile: 0:31:53.216755
40th percentile: 1:54:09.209372
50th percentile: 9:06:35.217203
75th percentile: 1 day, 23:25:00.212587
90th percentile: 3 days, 20:35:56.203524
99th percentile: 146 days, 23:09:57.219556


In [59]:
print(min(how_long_ago))
print(max(how_long_ago))

0:31:34.204549
287 days, 0:47:57.218849


##### size of response 

In [ ]:
import sys

sys.getsizeof(str(response.json()["result"]))

240652

That is around 0.25MB per request

Daily (with request every 10 seconds) 6 * 60 * 24 * 0.25 = 2160MB 

approximately **2GB of raw data per day**